# Terminal Processing

© Crown-owned copyright 2024, Defence Science and Technology Laboratory UK

This notebook serves as a guide on the functionality and use of the new Terminal simulation component.

By default, the Terminal will come pre-installed on any simulation component which inherits from `HostNode` (Computer, Server, Printer), and simulates the Secure Shell (SSH) protocol as the communication method.

In [ ]:
from primaite.simulator.system.services.terminal.terminal import Terminal
from primaite.simulator.network.container import Network
from primaite.simulator.network.hardware.nodes.host.computer import Computer
from primaite.simulator.system.applications.red_applications.ransomware_script import RansomwareScript

In [ ]:
def basic_network() -> Network:
    """Utility function for creating a default network to demonstrate Terminal functionality"""
    network = Network()
    node_a = Computer(hostname="node_a", ip_address="192.168.0.10", subnet_mask="255.255.255.0", start_up_duration=0)
    node_a.power_on()
    node_b = Computer(hostname="node_b", ip_address="192.168.0.11", subnet_mask="255.255.255.0", start_up_duration=0)
    node_b.power_on()
    network.connect(node_a.network_interface[1], node_b.network_interface[1])
    return network

The terminal can be accessed from a `HostNode` via the `software_manager` as demonstrated below. 

In the example, we have a basic network consisting of two computers 

In [ ]:
network: Network = basic_network()
computer_a: Computer = network.get_node_by_hostname("node_a")
terminal_a: Terminal = computer_a.software_manager.software.get("Terminal")
computer_b: Computer = network.get_node_by_hostname("node_b")
terminal_b: Terminal = computer_b.software_manager.software.get("Terminal")


To be able to send commands from `node_a` to `node_b`, you will need to `login` to `node_b` first, using valid user credentials. In the example below, we are logging in to the 'admin' account on `node_b`. 
If you are not logged in, any commands sent will be rejected.

In [ ]:
# Login to the remote (node_b) from local (node_a)
from primaite.simulator.system.services.terminal.terminal import RemoteTerminalConnection


term_a_term_b_remote_connection: RemoteTerminalConnection = terminal_a.login(username="admin", password="Admin123!", ip_address=computer_b.network_interface[1].ip_address)

In [ ]:
computer_b.software_manager.show()

In [ ]:
print(type(term_a_term_b_remote_connection))
term_a_term_b_remote_connection.execute(["software_manager", "application", "install", "RansomwareScript"])

In [ ]:
computer_b.software_manager.show()

You can view all remote connections to a terminal through use of the `show()` method

In [ ]:
terminal_b.show()

The Terminal can be used to send requests to install new software. The code block below demonstrates how the Terminal class allows the user of `terminal_a`, on `computer_a`, to send a command to `computer_b` to install the `RansomwareScript` application. 


The below example shows how you can send a command via the terminal to create a folder on the target Node.

Here, we send a command to `computer_b` to create a new folder titled "Downloads".

The resultant call to `computer_b.file_system.show()` shows that the new folder has been created.